ASL, v.0902323

# Introduction

In this notebook, we use Transformers on the same task as in the previous notebook (`DAT255-NLP-2.0-MedTweets-fastai-ULMFiT.ipynb`)

# Setup

In [1]:
# This is a quick check of whether the notebook is currently running on Google Colaboratory
# or on Kaggle, as that makes some difference for the code below.
# We'll do this in every notebook of the course.
try:
    import colab
    colab=True
except:
    colab=False

import os
kaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [2]:
%matplotlib inline
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path

In [3]:
#import os
#os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [4]:
if (colab or kaggle):
    !pip install transformers datasets

In [5]:
if colab:
    from google.colab import drive
    drive.mount("/content/gdrive")
    DATA = Path("/content/gdrive/MyDrive/Colab Notebooks/dat255-data")
    DATA.mkdir(exist_ok=True)
if not colab:
    DATA=Path('./data')
    DATA.mkdir(exist_ok=True)

In [6]:
import torch

In [7]:
import datasets

In [8]:
from datasets import load_dataset

In [9]:
import transformers

In [10]:
# Verify that the transformers library is installed and operational
print(transformers.pipeline('sentiment-analysis')('this is great!'))

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998694658279419}]


In [11]:
from datasets import Dataset
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, 
                          PreTrainedModel, BertModel, BertForSequenceClassification,
                          TrainingArguments, Trainer, GPT2ForSequenceClassification, 
                          RobertaForSequenceClassification)

from transformers.modeling_outputs import SequenceClassifierOutput

# MedWeb using Transformers

Load the data as before:

In [12]:
df = pd.read_csv('https://github.com/HVL-ML/DAT255/raw/main/3-NLP/data/medwebdata.csv')
df.head()

,ID,Tweet,Influenza,Diarrhea,Hayfever,Cough,Headache,Fever,Runnynose,Cold,labels,is_test
0,1en,The cold makes my whole body weak.,0,0,0,0,0,0,0,1,Cold,False
1,2en,It's been a while since I've had allergy sympt...,0,0,1,0,0,0,1,0,Hayfever;Runnynose,False
2,3en,I'm so feverish and out of it because of my al...,0,0,1,0,0,1,1,0,Hayfever;Fever;Runnynose,False
3,4en,"I took some medicine for my runny nose, but it...",0,0,0,0,0,0,1,0,Runnynose,False
4,5en,I had a bad case of diarrhea when I traveled t...,0,0,0,0,0,0,0,0,sober,False


For convenience, we combine all the labels into one vector stored under `y`:

In [13]:
df.drop(['is_test','labels'], axis=1, inplace=True)

In [14]:
df['labels'] = df.apply(lambda x: [x[c] for c in df.columns[2:]], axis=1)

In [15]:
df.head()

,ID,Tweet,Influenza,Diarrhea,Hayfever,Cough,Headache,Fever,Runnynose,Cold,labels
0,1en,The cold makes my whole body weak.,0,0,0,0,0,0,0,1,"[0, 0, 0, 0, 0, 0, 0, 1]"
1,2en,It's been a while since I've had allergy sympt...,0,0,1,0,0,0,1,0,"[0, 0, 1, 0, 0, 0, 1, 0]"
2,3en,I'm so feverish and out of it because of my al...,0,0,1,0,0,1,1,0,"[0, 0, 1, 0, 0, 1, 1, 0]"
3,4en,"I took some medicine for my runny nose, but it...",0,0,0,0,0,0,1,0,"[0, 0, 0, 0, 0, 0, 1, 0]"
4,5en,I had a bad case of diarrhea when I traveled t...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0]"


Set up the transformers model. There are multiple possible models to try (at the time of writing, HuggingFace has 146,394 models in its library). 

One interesting option is the [PubMedBERT model](https://huggingface.co/microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract) created by Microsoft Research by training a BERT model on 14 million abstracts of PubMed articles. Have a look at the blog post [Domain-specific language model pretraining for biomedical natural language processing](https://www.microsoft.com/en-us/research/blog/domain-specific-language-model-pretraining-for-biomedical-natural-language-processing/) and the accompanying paper. 

A related model is the BioMed-RoBERTa model from AllenAI: https://huggingface.co/allenai/biomed_roberta_base. 

The more recent GPT models are also interesting options (for example the BioMedLM model from Stanford CRFM: https://huggingface.c/stanford-crfm/BioMedLM). Unfortunately, GPT models require enormous amounts of computing resources compared to many alternatives. See the end of the notebook for an example run of the GPT-2 model BioMedLM. 

In [16]:
#model_name = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract'
#model_name = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"

model_name = "allenai/biomed_roberta_base"

model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

Some weights of the model checkpoint at allenai/biomed_roberta_base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at allenai/biomed_roberta_base and are newly initialized: ['classi

We need to tokenize the data in the same way as was done for the original dataset and create a data set compatible with HuggingFace:

In [17]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [18]:
ds = Dataset.from_pandas(df, split='train').train_test_split()

In [19]:
ds

DatasetDict({
    train: Dataset({
        features: ['ID', 'Tweet', 'Influenza', 'Diarrhea', 'Hayfever', 'Cough', 'Headache', 'Fever', 'Runnynose', 'Cold', 'labels'],
        num_rows: 1920
    })
    test: Dataset({
        features: ['ID', 'Tweet', 'Influenza', 'Diarrhea', 'Hayfever', 'Cough', 'Headache', 'Fever', 'Runnynose', 'Cold', 'labels'],
        num_rows: 640
    })
})

In [20]:
ds['train'][0]

{'ID': '889en',
 'Tweet': "My brother says he has the flu, and he's been vomitting since last night.",
 'Influenza': 0,
 'Diarrhea': 0,
 'Hayfever': 0,
 'Cough': 0,
 'Headache': 0,
 'Fever': 0,
 'Runnynose': 0,
 'Cold': 0,
 'labels': [0, 0, 0, 0, 0, 0, 0, 0]}

In [21]:
def tokenize_and_encode(examples):
    return tokenizer(examples["Tweet"], truncation=True)

In [22]:
cols = ds['train'].column_names
cols.remove('labels')
ds_enc = ds.map(tokenize_and_encode, batched=True, remove_columns=cols)

Map:   0%|          | 0/1920 [00:00<?, ? examples/s]

Map:   0%|          | 0/640 [00:00<?, ? examples/s]

In [23]:
ds_enc

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 1920
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 640
    })
})

In [26]:
model_name

'allenai/biomed_roberta_base'

In [27]:
num_labels=8

if "roberta" not in model_name:
    print("Assuming a BERT model")
    model = BertForSequenceClassification.from_pretrained(model_name, 
                                                        problem_type="multi_label_classification", 
                                                        num_labels=num_labels)

elif "roberta" in model_name:
    print("Assuming a RoBERTa model")
    model = RobertaForSequenceClassification.from_pretrained(model_name, 
                                                        problem_type="multi_label_classification", 
                                                        num_labels=num_labels)

Assuming a RoBERTa model


Some weights of the model checkpoint at allenai/biomed_roberta_base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at allenai/biomed_roberta_base and are newly initialized: ['classi

We define some metrics to use when scoring on the test data:

In [28]:
from sklearn.metrics import f1_score
def accuracy_thresh(y_pred, y_true, thresh=0.5, sigmoid=True): 
    y_pred = torch.from_numpy(y_pred)
    y_true = torch.from_numpy(y_true)
    if sigmoid: 
        y_pred = y_pred.sigmoid()
    return ((y_pred>thresh)==y_true.bool()).float().mean().item()

def f1score_thresh(y_pred, y_true, average='micro',thresh=0.5, sigmoid=True): 
    y_pred = torch.from_numpy(y_pred)
    y_true = torch.from_numpy(y_true)
    if sigmoid: 
        y_pred = y_pred.sigmoid()
    return f1_score(y_true, y_pred>thresh, average=average)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return {'accuracy_thresh': accuracy_thresh(predictions, labels),
           'f1score_micro_thresh': f1score_thresh(predictions, labels, average='micro'),
           'f1score_macro_thresh': f1score_thresh(predictions, labels, average='macro')}

..and then the training setup:

In [29]:
batch_size = 8
num_train_epochs = 6

args = TrainingArguments(
    output_dir=".",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    logging_steps=50,
    weight_decay=0.01, 
    save_steps=2000
)

We have to modify the loss function to deal with multilabel problems. Here's a way to do it (from https://discuss.huggingface.co/t/fine-tune-for-multiclass-or-multilabel-multiclass/4035/9):

In [30]:
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.BCEWithLogitsLoss()
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), 
                        labels.float().view(-1, self.model.config.num_labels))
        return (loss, outputs) if return_outputs else loss

In [31]:
trainer = MultilabelTrainer(
    model,
    args,
    train_dataset=ds_enc["train"],
    eval_dataset=ds_enc["test"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)

Let's see how the model does without any training on the MedWeb data:

In [32]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 640
  Batch size = 8
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.6963170766830444,
 'eval_accuracy_thresh': 0.6019531488418579,
 'eval_f1score_micro_thresh': 0.20203602192638997,
 'eval_f1score_macro_thresh': 0.20203602192638997,
 'eval_runtime': 2.0412,
 'eval_samples_per_second': 313.538,
 'eval_steps_per_second': 39.192}

Then fine-tune it:

In [33]:
trainer.train()

/home/alex/anaconda3/envs/huggingface/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1920
  Num Epochs = 6
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1440
  Number of trainable parameters = 124651784


Epoch,Training Loss,Validation Loss,Accuracy Thresh,F1score Micro Thresh,F1score Macro Thresh
1,0.207000,0.155405,0.958984,0.839204,0.839204
2,0.107800,0.103482,0.973828,0.892971,0.892971
3,0.078000,0.094105,0.974609,0.897959,0.897959
4,0.055800,0.090626,0.975000,0.899213,0.899213
5,0.044600,0.091549,0.975586,0.900557,0.900557
6,0.043100,0.089094,0.975781,0.901743,0.901743


***** Running Evaluation *****
  Num examples = 640
  Batch size = 8
***** Running Evaluation *****
  Num examples = 640
  Batch size = 8
***** Running Evaluation *****
  Num examples = 640
  Batch size = 8
***** Running Evaluation *****
  Num examples = 640
  Batch size = 8
***** Running Evaluation *****
  Num examples = 640
  Batch size = 8
***** Running Evaluation *****
  Num examples = 640
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1440, training_loss=0.1102306067943573, metrics={'train_runtime': 112.0431, 'train_samples_per_second': 102.818, 'train_steps_per_second': 12.852, 'total_flos': 176733816303744.0, 'train_loss': 0.1102306067943573, 'epoch': 6.0})

## How does it compare to other approaches?

From the [original article](https://www.jmir.org/2019/2/e12783/) from 2019 that presented the data set:

<img src="https://github.com/MMIV-ML/ELMED219-2022/raw/main/Lab2-NLP/assets/medweb_results.png">

The "NAIST-en" models are _"ensembles of hierarchical attention network and deep character-level convolutional neural network with loss functions (negative loss function, hinge, and hinge squared)"_. I.e. also deep learning-based models.

> **Your turn!** What about trying an ensemble of BERT or RoBERTa models? 

# Extra: using a GPT-2 model

> The following computations were done on a more powerful computer, equipped with an NVIDIA RTX A6000, 48GB GPU.

In [16]:
model_name = "stanford-crfm/BioMedLM" # This is a GPT2 model
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

Some weights of the model checkpoint at stanford-crfm/BioMedLM were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at stanford-crfm/BioMedLM and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# GPT2
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model = GPT2ForSequenceClassification.from_pretrained(model_name, 
                                                        problem_type="multi_label_classification", 
                                                        num_labels=num_labels)

In [27]:
model.config.pad_token_id = model.config.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
batch_size = 1
num_train_epochs = 2



In [ ]:
args = TrainingArguments(
    output_dir=".",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    logging_steps=50,
    weight_decay=0.01
)

In [ ]:
trainer = MultilabelTrainer(
    model,
    args,
    train_dataset=ds_enc["train"],
    eval_dataset=ds_enc["test"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)

In [36]:
# GPT2
trainer.train()

/home/alex/anaconda3/envs/huggingface/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1920
  Num Epochs = 2
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 3840
  Number of trainable parameters = 2594268160
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy Thresh,F1score Micro Thresh,F1score Macro Thresh
1,0.139600,0.256822,0.951563,0.797054,0.797054
2,0.164600,0.163555,0.966211,0.865579,0.865579


Saving model checkpoint to ./checkpoint-500
Configuration saved in ./checkpoint-500/config.json
The model is bigger than the maximum size per checkpoint (10GB) and is going to be split in 2 checkpoint shards. You can find where each parameters has been saved in the index located at ./checkpoint-500/pytorch_model.bin.index.json.
tokenizer config file saved in ./checkpoint-500/tokenizer_config.json
Special tokens file saved in ./checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./checkpoint-1000
Configuration saved in ./checkpoint-1000/config.json
The model is bigger than the maximum size per checkpoint (10GB) and is going to be split in 2 checkpoint shards. You can find where each parameters has been saved in the index located at ./checkpoint-1000/pytorch_model.bin.index.json.
tokenizer config file saved in ./checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./checkpoint-1500
Configurat

TrainOutput(global_step=3840, training_loss=0.2673093371093273, metrics={'train_runtime': 2015.1392, 'train_samples_per_second': 1.906, 'train_steps_per_second': 1.906, 'total_flos': 1032739377500160.0, 'train_loss': 0.2673093371093273, 'epoch': 2.0})